In [1]:
import pandas as pd
import re


In [2]:
df = pd.read_csv("item_description.csv")  # ganti sesuai nama file lu


In [3]:
df.columns


Index(['Item Description'], dtype='object')

In [4]:
def clean_text(x):
    if pd.isna(x):
        return x

    x = str(x)

    # lowercase (biar konsisten)
    x = x.lower()

    # hapus invisible char (newline, tab, NBSP)
    x = re.sub(r'[\n\r\t\xa0]', ' ', x)

    # hapus slash & backslash
    x = re.sub(r'[\\/]', ' ', x)

    # hapus karakter aneh (selain huruf, angka, spasi)
    x = re.sub(r'[^a-z0-9\s]', ' ', x)

    # rapihin spasi
    x = re.sub(r'\s+', ' ', x).strip()

    return x


In [5]:
df['item_desc_clean'] = df['Item Description'].apply(clean_text)


In [6]:
df.head()

,Item Description,item_desc_clean
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Item Description  861 non-null    object
 1   item_desc_clean   861 non-null    object
dtypes: object(2)
memory usage: 13.6+ KB


In [8]:
df_duplikat = df[df.duplicated(subset=['item_desc_clean'], keep=False)]
df_duplikat.sort_values('item_desc_clean').head(30)


,Item Description,item_desc_clean
105,"Dismantle,Dismantle Antennas 1.8m,Per Hop",dismantle dismantle antennas 1 8m per hop
106,"Dismantle,Dismantle Antennas >1.8m,Per Hop",dismantle dismantle antennas 1 8m per hop
526,"Installation,Install Antennas 1.8m,Per Hop",installation install antennas 1 8m per hop
528,"Installation,Install Antennas >1.8m,Per Hop",installation install antennas 1 8m per hop
720,"TSEL service package 2025 JABO<= 4 points, per...",tsel service package 2025 jabo 4 points per point
721,"TSEL service package 2025 JABO> 4 points, per ...",tsel service package 2025 jabo 4 points per point


In [9]:
def get_categorization_L1(item_desc_clean):
    """
    Analyzes an item description and returns a dictionary with 4 levels of categorization:
    1. Service Type
    2. Level 2 Category (Material/Service)
    3. Level 3 Category (Equipment Type)
    4. Level 4 Unit
    """
    text = str(item_desc_clean).lower()
    
    # ==============================================================================
    # LEVEL 1: SERVICE TYPE
    # Segregates into: TIN, RNO, MSS, ENG, IBC
    # ==============================================================================
    
    def get_service_type(t):
        # 1. Specific User Overrides
        if "netgear re-tagging works on tower" in t: return 'Operation & Maintenance (MSS)'
        if "splicing and tracing core in existing closure" in t: return 'Operation & Maintenance (MSS)'
        if " ms " in t or t.startswith("ms ") or t.endswith(" ms"): return 'Operation & Maintenance (MSS)'
        if "rigger" in t: return 'Optimization & Network Management (RNO)'
        if "for new build" in t: return 'Installation-Test-Commissioning (TIN)'

        # 2. Inbuilding Coverage Services (IBC)
        # Specific tech keywords only. Generic "indoor" is excluded to avoid false positives.
        if any(k in t for k in ['inbuilding', 'lampsite', 'ibs', 'das', 'small cell', 
                                'repeater', 'prru', 'rhub', 'indoor antenna']):
            return 'Inbuilding Coverage Services (IBC)'
            
        # 3. Optimization & Network Management (RNO)
        if any(k in t for k in ['optimization', 'optimize', 'drive test', 'walk test', 
                                'network performance', 'kpi', 'benchmark', 'npo', 'parameter', 
                                'tuning', 'measurement', 'ssv', 'rno', 'rf adjustment', 
                                'data collection', 'oss data', 'analysis', 'npi']):
            return 'Optimization & Network Management (RNO)'
        
        # 4. Survey (ENG)
        if any(k in t for k in ['survey', 'tssr', 'site audit', 'design', 'planning', 
                                'los', 'site hunt', 'sid', 'drawing', 'desgin', 'boq', 'tagging', 
                                'gps coordinate']):
            return 'Survey (ENG)'
        
        # 5. Operation & Maintenance (MSS)
        if any(k in t for k in ['maintenance', 'helpdesk', 'managed service', 'operation', 
                                'repair', 'spare part', 'fault', 'fuel', 'genset', 'cleaning', 
                                'rent', 'lease', 'bill', 'support', 'corrective', 'preventive', 
                                'ms fee', 'monitoring', 'assurance', 'keeper', 'freon', 
                                'refrigerant', 'rewinding', 'taskforce', 'site visit', 
                                'good part', 'handling adhoc', 'purifikasi', 'bushing', 
                                'contactor', 'oli', 'capacity allowance', 'seal', 'pressure suit']):
            return 'Operation & Maintenance (MSS)'
        
        # 6. Default Fallback (TIN)
        return 'Installation-Test-Commissioning (TIN)'
    return {
        "Level 1 Service Type": get_service_type(text)
    }


In [10]:
result = df['item_desc_clean'].apply(get_categorization_L1)

result_df = pd.json_normalize(result)

df = pd.concat([df, result_df], axis=1)


In [11]:
df.head()

,Item Description,item_desc_clean,Level 1 Service Type
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN)
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN)
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN)
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN)
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN)


In [12]:
df['Level 1 Service Type'].nunique()


5

In [13]:
df['Level 1 Service Type'].value_counts()


Level 1 Service Type
Installation-Test-Commissioning (TIN)      547
Operation & Maintenance (MSS)              139
Optimization & Network Management (RNO)    126
Survey (ENG)                                28
Inbuilding Coverage Services (IBC)          21
Name: count, dtype: int64

In [14]:
df[df['Level 1 Service Type'] == 'Installation-Test-Commissioning (TIN)']


,Item Description,item_desc_clean,Level 1 Service Type
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN)
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN)
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN)
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN)
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN)
...,...,...,...
827,XLSmart_service_package_2025_Non_JABO(WJ_SS_CS...,xlsmart service package 2025 non jabo wj ss cs...,Installation-Test-Commissioning (TIN)
838,XL_GUL_RF_IBC WT&OSS for Pre-Modernization,xl gul rf ibc wt oss for pre modernization,Installation-Test-Commissioning (TIN)
854,foundation Pole for aerial cable include asseo...,foundation pole for aerial cable include asseo...,Installation-Test-Commissioning (TIN)
855,galvanized Pole for aerial cable: 7m.,galvanized pole for aerial cable 7m,Installation-Test-Commissioning (TIN)


In [15]:
def clean_text_2(x):
    if pd.isna(x):
        return x

    x = str(x)

    # lowercase (biar konsisten)
    x = x.lower()
    # hapus invisible char (newline, tab, NBSP)
    x = re.sub(r'[\n\r\t\xa0]', ' ', x)
    # rapihin spasi
    x = re.sub(r'\s+', ' ', x).strip()
   
    return x


In [16]:
df['item_desc_clean_L2'] = df['Item Description'].apply(clean_text_2)


In [17]:
df.head()

,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN),"1 hdpe pipe 20mm od, 2mm wt"
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN),"1 hdpe pipe 32mm od, 2mm w t"
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN),1-3 aau installation(not on tower)-/site
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru dismantle (rru not on tower)-/site
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru installation(rru not on tower)-/site


In [18]:
df_duplikat = df[df.duplicated(subset=['item_desc_clean_L2'], keep=False)]
df_duplikat.sort_values('item_desc_clean_L2').head(30)


,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2


In [19]:
def get_categorization_L2(item_desc_clean_L2):
    """
    Analyzes an item description and returns a dictionary with 4 levels of categorization:
    1. Service Type
    2. Level 2 Category (Material/Service)
    3. Level 3 Category (Equipment Type)
    4. Level 4 Unit
    """
    text = str(item_desc_clean_L2).lower()
    # ==============================================================================
    # LEVEL 2: SERVICE VS MATERIAL
    # Segregates into: Services Only, Material Supply, Material & Services
    # ==============================================================================

    def get_level_2(t):
        # Priority 1: Material & Services (Combo items)
        if any(k in t for k in ["supply & install", "supply and install", "supply&install",
                                "provide & install", "provide and install", "supply & fix", 
                                "supply and fix", "including wiring", "include wiring"]):
            return "Material & Services"
        
        # Priority 2: Services Only (Action verbs and service tasks)
        service_keywords = [
            "installation", " install", "install ", "dismantle", "service", "survey", 
            "optimize", "optimization", "maintenance", "manpower", "team", "visit", 
            "test", "commissioning", "integration", "configuration", "audit", "design", 
            "training", "consultancy", "logistic", "transport", "mobilization", "incentive", 
            "return", "handling", "tagging", "splicing", "trench", "pulling", "roding", 
            "construct", "drilling", "rewinding", "unwelding", "drawing", "documentation", 
            "assurance", "monitoring", "analysis", "reporting", "tuning", "clearance", 
            "keeper", "helpdesk", "managed service", "support", "verification", "collection", 
            "induction", "registration", "rectification", "relocation", "replacement", 
            "replace", "swap", "expansion", "upgrade", "implementation", "migration", 
            "re-tagging", "trouble shooting", "cleaning", "rent", "lease", "bill", 
            "purifikasi", "moving", "codeploy", "co-deploy", "works", "work",
            "fusion", "los survey", "parameter", "adjustment", "rigger", "custimization",
            "customization", "supervision", "atp", "bastian", "bast", "acceptance",
            "compensation", "pick up", "fee", "allowance", "permit", "po for",
            "document", "welding", "taskforce", "checking", "check", "oss", "walk test", 
            "wt&oss", "pre-modernization", "modernization", "laying", "termination", 
            "rearrangement", "pm package"
        ]
        if any(k in t for k in service_keywords):
            return "Services Only"
        if t.startswith("new:") or t.startswith("new-") or t.startswith("new "):
            return "Services Only"
            
        # Priority 3: Material Supply (Default)
        return "Material Supply"

    # --- EXECUTE CATEGORIZATION ---
    return {
        "Level 2 Category": get_level_2(text)
    }


In [20]:
result = df['item_desc_clean_L2'].apply(get_categorization_L2)

result_df = pd.json_normalize(result)

df = pd.concat([df, result_df], axis=1)


In [21]:
df.head()

,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2,Level 2 Category
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN),"1 hdpe pipe 20mm od, 2mm wt",Material Supply
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN),"1 hdpe pipe 32mm od, 2mm w t",Material Supply
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN),1-3 aau installation(not on tower)-/site,Services Only
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru dismantle (rru not on tower)-/site,Services Only
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru installation(rru not on tower)-/site,Services Only


In [22]:
df['Level 2 Category'].nunique()


3

In [23]:
df['Level 2 Category'].value_counts()


Level 2 Category
Services Only          747
Material Supply        103
Material & Services     11
Name: count, dtype: int64

In [24]:
def get_categorization_L3(item_desc_clean):
    """
    Analyzes an item description and returns a dictionary with 4 levels of categorization:
    1. Service Type
    2. Level 2 Category (Material/Service)
    3. Level 3 Category (Equipment Type)
    4. Level 4 Unit
    """
    text = str(item_desc_clean).lower()
    # ==============================================================================
    # LEVEL 3: EQUIPMENT TYPE
    # Segregates based on domain (RAN, TRM, OSP, Power, etc.)
    # ==============================================================================

    def get_level_3(t):
        # 1. Incentive (Highest Priority)
        if "incentive" in t or "points" in t:
            return "Incentive"

        # 2. Logistic
        if "logistic" in t or "return fault" in t or "good part" in t:
            return "Logistic"

        # 3. Outside Plant (OSP)
        # Explicit override for cable/trenching works
        if any(k in t for k in ["pulling", "trench", "cable", "otb"]):
            return "Outside Plant (OSP)"

        # 4. Power System
        # Specific power equipment (Rectifier, Battery, etc.)
        if any(k in t for k in ["power", "rectifier", "battery", "genset", "generator", 
                                "ups", "inverter", "kwh", "mcb", "transformer", 
                                "ac power", "dc power", "lv ", "hv ", "breaker", "busbar"]):
            return "Power System"

        # 5. Radio Access Network (RAN)
        # Includes GUL, BTS, RRU, SOC, Optimization items
        if any(k in t for k in ["ran", "radio", "bts", "nodeb", "enodeb", "gnodeb", 
                                "sran", "base station", "rru", "aau", "bbu", "rfu", "trx", 
                                "sector", "cell ", "cell-", "multi-sector", "antenna", 
                                "feeder", "jumper", "cpri", "tma", "combiner", "diplexer", 
                                "triplexer", "filter", "4g", "5g", "lte", "gul", "gsm", 
                                "umts", "nr ", "nsa", " sa ", "drive test", "walk test", 
                                "dt ", "ssv", "sso", "npi", "rno", "rf adjustment", 
                                "parameter", "optimization", "tuning", "network performance", 
                                "kpi", "rigger", "inbuilding", "ibs", "das", "lampsite", 
                                "small cell", "repeater", "prru", "rhub", "mocn", "roaming", 
                                "neighbor", "blind spot", "coverage", "distribution aerial", 
                                "feeder aerial", "service operation center", "remote radio unit"]):
            return "Radio Access Network (RAN)"

        # 6. Transmission (TRM)
        if any(k in t for k in ["microwave", " mw ", " mw-", "-mw", "odu", "idu", 
                                "transmission", "trm", "backhaul", "per hop", "optical", 
                                "fiber", "fibre", " fo ", "ftth", "fbb", "osp", "isp", 
                                "olt", "ont", "gpon", "wdm", "mstp", "sdh", "dwdm", 
                                "otn", "ptn", "datacomm", "router", "switch", "ip ne", 
                                "ip_ne", "peering", "splicing", "roding", "duct", "aerial", 
                                "underground", "odf", "patch cord", "fat ", "fdt ", "closure",
                                "lan ", "cat 6", "cat6", "metro", "ethernet", "access,olt",
                                "access,blade", "access,rack", "wdm/mstp"]):
            return "Transmission (TRM)"

        # 7. Core Network (CORE)
        if any(k in t for k in ["core network", " msc", " hlr", " hss", " epc", " mme", 
                                " sgsn", " ggsn", " ims", " volte", "cs core", "ps core", 
                                "packet core", "udm", "ausf", "amf", "smf", "upf", "pcrf", 
                                "dra", "stp", "sbc", "mgw", "softswitch", "media gateway", 
                                "signaling", "user plane", "control plane", "charging system", 
                                "ocs", "pcc", "cs helpdesk", "core site"]):
            return "Core Network (CORE)"

        # 8. Supporting Facility (CME)
        # Civil, Mechanical, Facility items not caught by Power System
        if any(k in t for k in ["cme", "ac ", "dc ", "air conditioner", "conditioning", 
                                "pac ", "split ac", "cooling", "cabinet", "shelter", 
                                "enclosure", "rack", "cage", "pole", "tower", "monopole", 
                                "guyed", "civil", "grounding", "lightning", "arrester", 
                                "protection", "fuel", "tank", "sensor", "mechanical", 
                                "electrical", "concrete", "foundation", "fence", "tray", 
                                "ladder", "bracket", "mounting", "clamp", "bolt", "nut", 
                                "anchor", "pipe", "conduit", "trunking", "seal", "cement", 
                                "sand", "gravel", "macadam", "renovation", "refurbishment", 
                                "cleaning", "site keeper", "security", "freon", "oil", 
                                "lubricant", "pump", "valve", "compressor", "bushing", 
                                "contactor", "mecanical", "pressure suit", "reinstatement", 
                                "welding", "drilling", "hole", "anti theft", "atd", "lug", 
                                "scun", "connector"]):
            return "Supporting Facility (CME)"
            
        # 9. General Support (Default)
        return "General Support"

    # --- EXECUTE CATEGORIZATION ---
    return {
        "Level 3 Category": get_level_3(text)
    }

In [25]:
result = df['item_desc_clean'].apply(get_categorization_L3)

result_df = pd.json_normalize(result)

df = pd.concat([df, result_df], axis=1)


In [26]:
df.head()

,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2,Level 2 Category,Level 3 Category
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN),"1 hdpe pipe 20mm od, 2mm wt",Material Supply,Supporting Facility (CME)
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN),"1 hdpe pipe 32mm od, 2mm w t",Material Supply,Supporting Facility (CME)
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN),1-3 aau installation(not on tower)-/site,Services Only,Radio Access Network (RAN)
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru dismantle (rru not on tower)-/site,Services Only,Radio Access Network (RAN)
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru installation(rru not on tower)-/site,Services Only,Radio Access Network (RAN)


In [27]:
df['Level 3 Category'].nunique()


9

In [28]:
df['Level 3 Category'].value_counts()


Level 3 Category
Radio Access Network (RAN)    339
Logistic                      109
Transmission (TRM)            103
Power System                   85
Outside Plant (OSP)            73
General Support                64
Supporting Facility (CME)      40
Incentive                      36
Core Network (CORE)            12
Name: count, dtype: int64

In [29]:
def get_categorization_L4(item_desc_clean_L2):
    """
    Analyzes an item description and returns a dictionary with 4 levels of categorization:
    1. Service Type
    2. Level 2 Category (Material/Service)
    3. Level 3 Category (Equipment Type)
    4. Level 4 Unit
    """
    text = str(item_desc_clean_L2).lower()
    # ==============================================================================
    # LEVEL 4: UNIT
    # Extracts the unit of measurement
    # ==============================================================================

    def get_level_4(t):
        if "man-month" in t or "man month" in t or "*month" in t or " month" in t:
            return "Man-month"
        if "team" in t:
            return "Team"
        if "/site" in t or "per site" in t:
            return "Site"
        if "/hop" in t or "per hop" in t or " hop" in t:
            return "Hop"
        if "/each" in t or "pcs" in t or "per point" in t:
            return "Each"
        return "Each" # Default

    # --- EXECUTE CATEGORIZATION ---
    return {
        "Level 4 Unit": get_level_4(text)
    }


In [30]:
result = df['item_desc_clean_L2'].apply(get_categorization_L4)

result_df = pd.json_normalize(result)

df = pd.concat([df, result_df], axis=1)


In [31]:
df.head()

,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2,Level 2 Category,Level 3 Category,Level 4 Unit
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN),"1 hdpe pipe 20mm od, 2mm wt",Material Supply,Supporting Facility (CME),Each
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN),"1 hdpe pipe 32mm od, 2mm w t",Material Supply,Supporting Facility (CME),Each
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN),1-3 aau installation(not on tower)-/site,Services Only,Radio Access Network (RAN),Site
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru dismantle (rru not on tower)-/site,Services Only,Radio Access Network (RAN),Site
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru installation(rru not on tower)-/site,Services Only,Radio Access Network (RAN),Site


In [32]:
df['Level 4 Unit'].nunique()


5

In [33]:
df['Level 4 Unit'].value_counts()


Level 4 Unit
Each         725
Site          47
Man-month     36
Hop           31
Team          22
Name: count, dtype: int64

In [34]:
def get_categorization_L5(item_desc_clean):
    """
    Level 5 Deployment Type Classification (REFINED & MERGED)

    Merges:
    - Original L5 rules
    - Refined rules from secondary script
    - Priority override handling
    """

    text = str(item_desc_clean).lower()

    def get_level_5(t):

        # ==========================================================
        # 1️⃣ HARD EXCEPTION (Highest Priority)
        # ==========================================================
        # Business rule: special phrase must override everything
        if "rectification and expansion" in t:
            return "Swap/Replacement"


        # ==========================================================
        # 2️⃣ SUPPORT
        # ==========================================================
        support_keywords = [
            'per point', 'allowance', 'incentive',
            'return fault', 'return good part', 'good part',
            'compensation', 'logistic from', 'self pick up', 'handling',
            'rigger', 'visit', 'document', 'coordinate', 'gps',
            'aos', 'project service', 
            'access registration', 'induction', 'team induction', 
            'restricted mining', 'special event', 'assurance', 
            'npi', 'taskforce'
        ]
        if any(k in t for k in support_keywords):
            return "Support"


        # ==========================================================
        # 3️⃣ OPTIMIZATION & PERFORMANCE MANAGEMENT
        # ==========================================================
        optimization_keywords = [
            'optimization', 'performance', 'tuning', 'audit', 'ssv', 'sso',
            'dt', 'drive test', 'npx', 'network performance', 'kpi', 'monitor',
            'collection', 'verification', 'adjustment', 'acceptance', 'benchmark',
            'parameters and neighboring cell', 'analysis and report',
            'quality management', 'neighboring cell scripts', 'without car',
            'walk test',
            'iro_ethernet configuration', 'iro_fbb common', 'iro_mw idu',
            'tsel_gul_rf_rigger', 'xl_gul_merge_rf_rigger',
            'xl_gul_rf_rigger', 'ioh_gul_rf_rigger', 'otdr', 'fiber quality', 
            'board testing', 'antenna checking', 'light indicator', 'idu', 'odu', 'bbu', 'rru',
        ]
        if any(k in t for k in optimization_keywords):
            return "Optimization & Performance Management"


        # ==========================================================
        # 4️⃣ MAINTENANCE SERVICES
        # ==========================================================
        maintenance_keywords = [
            'maintenance', 'trouble shooting', 'troubleshooting', 'repair',
            'corrective', 'preventive', 'spare part', 'spare', 'rectification',
            'fixing', 'warranty', 'helpdesk', 'support', 'ticket', 'complain',
            'service operation center', 'soc', 'alarm clearance', 'tagging',
            'reinstatement', 'site keeper', 'managed service',
            'shopping list package', 'shopping list',
            'refrigerant', 'refrigerant freon', 'refrigerant r22', 'refrigerant r407c',
            'pressure suit', 'purifikasi', 'rewinding', 'rewelding',
            'split ac', 'trench & backfill', 'xl ms lumpsum shopping list', 
            'cell down', 'p1', 'unwelding'
        ]
        if any(k in t for k in maintenance_keywords):
            return "Maintenance Services"


        # ==========================================================
        # 5️⃣ SWAP / REPLACEMENT (GENERAL)
        # ==========================================================
        swap_keywords = [
            'swap', 'replace', 'replacement', 'migration', 'modernization',
            'relocation', 're-arrangement', 'rearrange', 're-configuration'
        ]
        if any(k in t for k in swap_keywords):
            return "Swap/Replacement"


        # ==========================================================
        # 6️⃣ DISMANTLING
        # ==========================================================
        dismantle_keywords = [
            'dismantle', 'dismantlement', 'removal', 'remove',
            'de-installation', 'return'
        ]
        if any(k in t for k in dismantle_keywords):
            return "Dismantling"


        # ==========================================================
        # 7️⃣ ADDITIONAL / UPGRADE
        # ==========================================================
        upgrade_keywords = [
            'expansion', 'upgrade', 'capacity', 'add-on',
            'augment', 'growth', 'extension', 'license'
        ]
        if any(k in t for k in upgrade_keywords):
            return "Additional/Upgrade"


        # ==========================================================
        # 8️⃣ NEW DEPLOYMENT
        # ==========================================================
        deployment_keywords = [
            'new', 'installation', 'install', 'commissioning', 'integration',
            'survey', 'deployment', 'rollout', 'implement', 'implementation',
            'setup', 'construct', 'civil work', 'supply', 'material',
            'pipe', 'cable', 'pole', 'bracket', 'connector', 'adapter',
            'feeder', 'clamp', 'cabinet', 'rack', 'power', 'battery',
            'concrete', 'conduit', 'splicing', 'fusion', 'device',
            'drawing', 'dummy load', 'contactor',
            # hardware refine
            'jumper', 'kabel', 'mcb', 'rod', 'bushing', 'seal',
            'ties', 'module', 'lamp', 'trafo', 'mechanical',
            'oli', 'otb', 'patch cord', 'pipa', 'pvc',
            'tape', 'rhub box', 'codeploy', 'insert'
        ]
        if any(k in t for k in deployment_keywords):
            return "New Deployment"


        # ==========================================================
        # 9️⃣ FALLBACK
        # ==========================================================
        return "Others"


    return {
        "Level 5 Deployment Type": get_level_5(text)
    }


In [35]:
result = df['item_desc_clean'].apply(get_categorization_L5)

result_df = pd.json_normalize(result)

df = pd.concat([df, result_df], axis=1)


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Item Description         861 non-null    object
 1   item_desc_clean          861 non-null    object
 2   Level 1 Service Type     861 non-null    object
 3   item_desc_clean_L2       861 non-null    object
 4   Level 2 Category         861 non-null    object
 5   Level 3 Category         861 non-null    object
 6   Level 4 Unit             861 non-null    object
 7   Level 5 Deployment Type  861 non-null    object
dtypes: object(8)
memory usage: 53.9+ KB


In [37]:
df.head()

,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2,Level 2 Category,Level 3 Category,Level 4 Unit,Level 5 Deployment Type
0,"1 HDPE pipe 20mm OD, 2mm WT",1 hdpe pipe 20mm od 2mm wt,Installation-Test-Commissioning (TIN),"1 hdpe pipe 20mm od, 2mm wt",Material Supply,Supporting Facility (CME),Each,New Deployment
1,"1 HDPE pipe 32mm OD, 2mm W T",1 hdpe pipe 32mm od 2mm w t,Installation-Test-Commissioning (TIN),"1 hdpe pipe 32mm od, 2mm w t",Material Supply,Supporting Facility (CME),Each,New Deployment
2,1-3 AAU installation(not on tower)-/site,1 3 aau installation not on tower site,Installation-Test-Commissioning (TIN),1-3 aau installation(not on tower)-/site,Services Only,Radio Access Network (RAN),Site,New Deployment
3,1-3 RRU dismantle (RRU not on tower)-/Site,1 3 rru dismantle rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru dismantle (rru not on tower)-/site,Services Only,Radio Access Network (RAN),Site,Optimization & Performance Management
4,1-3 RRU installation(RRU not on tower)-/Site,1 3 rru installation rru not on tower site,Installation-Test-Commissioning (TIN),1-3 rru installation(rru not on tower)-/site,Services Only,Radio Access Network (RAN),Site,Optimization & Performance Management


In [38]:
df['Level 5 Deployment Type'].nunique()


8

In [39]:
df['Level 5 Deployment Type'].value_counts()


Level 5 Deployment Type
Support                                  262
New Deployment                           220
Optimization & Performance Management    212
Maintenance Services                      55
Dismantling                               43
Additional/Upgrade                        39
Swap/Replacement                          28
Others                                     2
Name: count, dtype: int64

In [40]:
df[df['Level 5 Deployment Type'] == 'Others']


,Item Description,item_desc_clean,Level 1 Service Type,item_desc_clean_L2,Level 2 Category,Level 3 Category,Level 4 Unit,Level 5 Deployment Type
325,IRO_Ethernet configuration from BTS/NodeB to B...,iro ethernet configuration from bts nodeb to b...,Installation-Test-Commissioning (TIN),iro_ethernet configuration from bts/nodeb to b...,Services Only,Radio Access Network (RAN),Each,Others
778,"Trench & Backfill works,50 cm depth, Normal Soil",trench backfill works 50 cm depth normal soil,Installation-Test-Commissioning (TIN),"trench & backfill works,50 cm depth, normal soil",Services Only,Outside Plant (OSP),Each,Others


In [41]:
# CSV
output_csv = "Gemini_L1-5.csv"
df.to_csv(output_csv, index=False)

In [41]:

# XLSX
output_xlsx = "Gemini_L5_Rev3.xlsx"
df.to_excel(output_xlsx, index=False)